In [48]:
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.gridspec as gridspec
from matplotlib.backends.backend_pdf import PdfPages
from tqdm import tqdm
import random
import os
import time
import scienceplots
plt.style.use(['science','ieee'])
'''
from matplotlib import rcParams
plt.rcParams['xtick.labelsize']  = 18
plt.rcParams['ytick.labelsize']  = 18
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family']      = 'STIXGeneral'
plt.rcParams['axes.labelsize']   = 20
plt.rcParams['font.size']        = 10
plt.rcParams['legend.fontsize']  = 20
plt.rcParams.update({'figure.autolayout': True})
'''

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Conv2DTranspose


import h5py
from tqdm import tqdm
from datetime import datetime


In [43]:
%%time
import auxLib; importlib.reload(auxLib)

Importing library...
Library successfully imported
Wall time: 50.7 ms


<module 'auxLib' from 'd:\\Facultad\\Doctorado\\Proyectos\\paper\\auxLib.py'>

In [49]:
L = 40

nros_densidades = ['055','06', '061', '062', '063', '064', '065', '07', '08', '09','1']

densidades_valor = [0.55, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.7, 0.8, 0.9, 1.0]

# Dense Neural Networks: Training 'all'

In [45]:
# All possible configurations

data = auxLib.SyntheticImagesGen(training=['all'], L=L)
train_images, train_labels = data.dataGenerator(10000)
test_images, test_labels = data.dataGenerator(1000)

train_images = train_images.reshape((10000, L * L))
test_images = test_images.reshape((1000, L * L))

Generating synthetic data...
Done!
Elapsed time: 0.7019157409667969 seconds
Generating synthetic data...
Done!
Elapsed time: 0.03128242492675781 seconds


In [46]:
# Dense neural network

dnn = auxLib.DenseNeuralNetworkGen((L*L,), 4, layers=[
    {'type': 'dense', 'units': 512, 'activation': 'relu'},
    {'type': 'dense', 'units': 4, 'activation': 'softmax'}
])

dnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
dnn.summary()
dnn.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 512)               819712    
                                                                 
 dense_5 (Dense)             (None, 4)                 2052      
                                                                 
Total params: 821764 (3.13 MB)
Trainable params: 821764 (3.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
313/313 [==============================] - 13s 35ms/step - loss: 0.0183 - accuracy: 0.9934 - val_loss: 3.2669e-04 - val_accuracy: 1.0000
Epoch 2/5
313/313 [==============================] - 12s 38ms/step - loss: 1.2659e-04 - accuracy: 1.0000 - val_loss: 1.8144e-04 - val_accuracy: 1.0000
Epoch 3/5
313/313 [==============================] - 13s 40ms/step - loss: 6.7627e-05 - accuracy: 1.0000 - val_loss:

In [54]:
# CALCULATE PREDICTIONS FUNCTION
def calculate_predictions(model, sim_images, reshape = True, verbose=True):
    if reshape:
        sim_images = [array.reshape((251,L*L)) for array in sim_images]
        
    prediction = model.predict(sim_images[0], verbose=verbose)/len(sim_images)
    for i in tqdm(range(1, len(sim_images)), 
                            desc="Predicting", unit="repetitions"):
        sim_im = sim_images[i]
        prediction += model.predict(sim_im, verbose=verbose)/len(sim_images)
    '''
    if I want to save the predictions:
    
    temperature = np.arange(0.0, 5.02, 0.02).tolist()
    combined_array = np.hstack((temperature, prediction))
    prediction_df = pd.DataFrame(combined_array, columns=['Temperature', 'Paramagnetic', 'Ferromagnetic', 'Neel', 'Stripe'])
    prediction_df.to_csv(os.path.join(os.getcwd(), 'models', f'predictions_{name}.csv'), index=False)
    '''
    return prediction

In [55]:
loader_init = auxLib.loader_and_saver(os.getcwd())

temperaturas_criticas = []

fig, axs = plt.subplots(4,3,figsize=(15,20))
fig.tight_layout() 

for index in range(11):

    sim_images, temperature = loader_init.simulatedImages(index)

    prediction = calculate_predictions(dnn, sim_images, reshape = True, verbose=False)
    
    ferro = prediction[:,1]
    para = prediction[:,0]
    row = index//3
    col = index%3
    axs[row, col].plot(temperature,ferro,label='Ferromagnetic')
    axs[row, col].plot(temperature,para,label='Paramagnetic')
    axs[row, col].grid()
    axs[row, col].set_title(r'Density $\rho$ '+f'= {densidades_valor[index]}')
    axs[row, col].set_xlabel('Temperature')
    axs[row, col].set_ylabel('Probability')

    if max(ferro)<0.5:
        tc = 0
    else:
        index_half = np.where(ferro > 0.5)[0][-1]
        # Interpolation:
        tc = temperature[index_half] + (ferro[index_half]-0.5)*(temperature[index_half+1] - temperature[index_half]) / (ferro[index_half] - ferro[index_half+1])
        axs[row, col].axvline(x=tc,  color = 'red', linestyle = '--', alpha = 0.5)

    temperaturas_criticas.append(tc)
    print(f'rho = {densidades_valor[index]}')
    print(f'T_c = {tc}')

axs[3, 2].axis('off')
fig.set_tight_layout(True)
lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
fig.legend(lines[:2], ('Ferromagnetic', 'Paramagnetic'), loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=3)
#plt.legend()
plt.show()

Loading simulated images...


Loading arrays: 100%|██████████| 196/196 [00:07<00:00, 25.32array/s]


Files loaded!
Data of density p = 0.55 succesfully loaded.


Predicting: 100%|██████████| 195/195 [00:46<00:00,  4.19repetitions/s]


rho = 0.55
T_c = 0
Loading simulated images...


...

rho = 0.9
T_c = 1.9593239657715356
Loading simulated images...


Loading arrays: 100%|██████████| 156/156 [00:04<00:00, 38.34array/s]


Files loaded!
Data of density p = 1. succesfully loaded.


Predicting: 100%|██████████| 155/155 [00:30<00:00,  5.01repetitions/s]


rho = 1.0
T_c = 2.3142841952910187


In [56]:
datadf = {"Densidad":densidades_valor,"Temperatura":temperaturas_criticas}

temp_df = pd.DataFrame(datadf)

temp_df.to_csv(os.path.join(os.getcwd(), 'models', 'temp_dnn_all.csv'), index=False)

In [62]:
densi = np.arange(0.0,1.002,0.002)
curva = densi*0
for i in range(len(densi)):
    if densi[i]<0.592746:
        curva[i] = 0
    elif densi[i]<0.89:
        curva[i] = -(0.983)/(np.log(densi[i]-0.592746))  
        last_i = i

for i in range(last_i, len(densi)):
     curva[i] = curva[last_i] + 1.7 * (densi[i]-densi[last_i])

plt.figure(figsize=(15,10))
plt.plot(temp_df['Densidad'],temp_df['Temperatura']/np.array(temp_df['Temperatura'])[-1],'x--')
plt.plot(densi,curva,'r')
plt.xlim(0.4,1)
plt.legend(temp_df.columns)
plt.show()

In [7]:
dnn.model_saver('dnn_all.h5')

C:\Users\Usuario\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
path = os.path.join(os.getcwd(), 'models', 'dnn_all.h5')

history = get_model